In [11]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, Flatten
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Model, load_model

import os
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Found GPU at: /device:GPU:0


In [2]:
with open('new_data.pickle', 'rb') as f:
    Data = pickle.load(f)
    Data = pd.DataFrame(Data)

# Train test split
idx = random.sample(range(2418), 480) # 80% for training
all_index = list(np.setdiff1d(range(2418), idx))

# Getting X_train, X_val, y_train and scale them

X_train = Data.iloc[all_index,:].drop(2251, axis = 1)
X_val = Data.iloc[idx,:].drop(2251, axis = 1)
y_train = pd.get_dummies(Data.iloc[all_index,:][2251])

# X_train = Data.iloc[all_index,:].drop('label', axis = 1)
# X_val = Data.iloc[idx,:].drop('label', axis = 1)
# y_train = pd.get_dummies(Data.iloc[all_index,:]['label'])
# print(y_train)
names = y_train.columns
mapping = {}
i = 0
for n in names:
    mapping[i] = n
    i+=1

# # scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [3]:
def identity_block(X, f, filters, stage, block):
    """
    实现图3的恒等块

    参数：
        X - 输入的tensor类型的数据，维度为( m, 1001， 1)
        f - 整数，指定主路径中间的CONV窗口的维度
        filters - 整数列表，定义了主路径每层的卷积层的过滤器数量
        stage - 整数，根据每层的位置来命名每一层，与block参数一起使用。
        block - 字符串，据每层的位置来命名每一层，与stage参数一起使用。

    返回：
        X - 恒等块的输出，tensor类型，维度为(1001，1)

    """

    #定义命名规则
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base   = "bn"  + str(stage) + block + "_branch"

    #获取过滤器
    F1, F2, F3 = filters

    #保存输入数据，将会用于为主路径添加捷径
    X_shortcut = X

    #主路径的第一部分
    ##卷积层
    X = Conv1D(filters=F1, kernel_size=1, strides=1 ,padding="valid",
               name=conv_name_base+"2a", kernel_initializer=glorot_uniform(seed=0))(X)
    ##归一化
    X = BatchNormalization(name=bn_name_base+"2a")(X)
    ##使用ReLU激活函数
    X = Activation("relu")(X)

    #主路径的第二部分
    ##卷积层
    X = Conv1D(filters=F2, kernel_size=f,strides=1, padding="same",
               name=conv_name_base+"2b", kernel_initializer=glorot_uniform(seed=0))(X)
    ##归一化
    X = BatchNormalization(name=bn_name_base+"2b")(X)
    ##使用ReLU激活函数
    X = Activation("relu")(X)


    #主路径的第三部分
    ##卷积层
    X = Conv1D(filters=F3, kernel_size=1, strides=1, padding="valid",
               name=conv_name_base+"2c", kernel_initializer=glorot_uniform(seed=0))(X)
    ##归一化
    X = BatchNormalization(name=bn_name_base+"2c")(X)
    ##没有ReLU激活函数

    #最后一步：
    ##将捷径与输入加在一起
    X = Add()([X,X_shortcut])
    ##使用ReLU激活函数
    X = Activation("relu")(X)

    return X

In [4]:
def convolutional_block(X, f, filters, stage, block, s=2):
    #定义命名规则
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base   = "bn"  + str(stage) + block + "_branch"

    #获取过滤器数量
    F1, F2, F3 = filters

    #保存输入数据
    X_shortcut = X
    
    #主路径
    ##主路径第一部分
    X = Conv1D(filters=F1, kernel_size=1, strides=s, padding="valid",
               name=conv_name_base+"2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2a")(X)
    X = Activation("relu")(X)

    ##主路径第二部分
    X = Conv1D(filters=F2, kernel_size=f, strides=1, padding="same",
               name=conv_name_base+"2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2b")(X)
    X = Activation("relu")(X)

    ##主路径第三部分
    X = Conv1D(filters=F3, kernel_size=1, strides=1, padding="valid",
               name=conv_name_base+"2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2c")(X)

    #捷径
    X_shortcut = Conv1D(filters=F3, kernel_size=1, strides=s, padding="valid",
               name=conv_name_base+"1", kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(name=bn_name_base+"1")(X_shortcut)

    #最后一步
    X = Add()([X,X_shortcut])
    X = Activation("relu")(X)

    return X

In [5]:
# Construct ResNet50
def ResNet50(input_shape=(2251,1),classes=186):
    #定义tensor类型的输入数据
    X_input = Input(input_shape)

    #0填充
    X = keras.layers.convolutional.ZeroPadding1D(3)(X_input)

    #stage1
    X = Conv1D(filters=64, kernel_size=7, strides=2, name="conv1",
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling1D(pool_size=3, strides=2)(X)

    #stage2
    X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block="a", s=1)
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="b")
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="c")

    #stage3
    X = convolutional_block(X, f=3, filters=[128,128,512], stage=3, block="a", s=2)
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="b")
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="c")
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="d")

    #stage4
    X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block="a", s=2)
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="b")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="c")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="d")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="e")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="f")

    #stage5
    X = convolutional_block(X, f=3, filters=[512,512,2048], stage=5, block="a", s=2)
    X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="b")
    X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="c")

    #均值池化层
    X = AveragePooling1D(pool_size=2,padding="same")(X)

    #输出层
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", name="fc"+str(classes),
              kernel_initializer=glorot_uniform(seed=0))(X)


    #创建模型
    cnn_model = Model(inputs=X_input, outputs=X, name="ResNet50")

    return cnn_model


In [12]:
cnn_model = ResNet50(input_shape=(2251,1),classes=186)
cnn_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
# Fitting CNN model
cnn_model.fit(X_train.reshape(1938, 2251, 1), y_train.values, batch_size = 32, epochs = 100, verbose = 1)

# Validating the CNN modela
pred = cnn_model.predict(X_val.reshape(480, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)

# Create category mapping dictionary
y_val = Data.iloc[idx,:][2251]
    
# Calculating accuracy
predicted = np.vectorize(mapping.get)(out)
print('Accuracy: '+ str(round(np.sum(predicted == y_val)/480,2) * 100) + '%')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1938/1938 [==============================] - 28s 14ms/step - loss: 15.7774 - acc: 0.0108
Epoch 2/100
1938/1938 [==============================] - 14s 7ms/step - loss: 15.8852 - acc: 0.0144
Epoch 3/100
1938/1938 [==============================] - 14s 7ms/step - loss: 15.8852 - acc: 0.0144
Epoch 4/100
1728/1938 [=========================>....] - ETA: 1s - loss: 15.9036 - acc: 0.0133

KeyboardInterrupt: 

In [13]:
# Load and test on the experimental data
with open('norm_T_origin.pickle', 'rb') as f:
    Test = pickle.load(f)
    Test = pd.DataFrame(Test)
X_test = scaler.transform(Test.iloc[:,:])
pred = cnn_model.predict(X_test.reshape(14, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)
# print(predicted)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:8]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['ZIF-90', 'MOF-5', 'ZIF-65', 'BAHGUN03', 'ZIF-8', 'AFIXES', 'HUYKUH', 'VOCQAF']
['ZIF-90', 'MOF-5', 'BAHGUN03', 'ZIF-65', 'BAHGUN01', 'UHAXUW', 'NICZAA01', 'VOCQAF']
['HUJVOX', 'LAMHAI', 'F-MOF-8', 'ZUVTEP', 'MIHBAG', 'LEJBUX', 'PUTYAE', 'JOSNAG']
['ZIF-90', 'MOF-5', 'ZIF-65', 'DEFKUU', 'BAHGUN03', 'ZIF-8', 'UHAXUW', 'BAHGUN01']
['PUTYEI', 'UGUQOC', 'ZIF-12', 'NUMMOX', 'MOF-5', 'WABVIF02', 'ZIF-71', 'UHAXUW']
['ZIF-12', 'ZIF-71', 'DEFKUU', 'PUTYEI', 'MIL-101', 'IDIWOH03', 'UGUQOC', 'ZIF-90']
['IDIWOH03', 'DEFKUU', 'ZIF-71', 'MIL-101', 'UGUQOC', 'ZIF-12', 'IDIWOH01', 'PUTYEI']
['ZIF-90', 'MOF-5', 'IDIWOH03', 'ZIF-65', 'ZIF-8', 'DEFKUU', 'JOSNAG01', 'BAHGUN03']
['MOF-5', 'ZIF-90', 'ZIF-8', 'IDIWOH03', 'ZIF-65', 'AFIXES', 'HUYKUH', 'UHAXUW']
['MOF-5', 'ZIF-90', 'ZIF-8', 'UHAXUW', 'AFIXES', 'HUYKUH', 'BAHGUN03', 'PUTYEI']
['MOF-5', 'ZIF-90', 'ZIF-8', 'IDIWOH03', 'UHAXUW', 'ZIF-65', 'AFIXES', 'HUYKUH']
['ZIF-90', 'MOF-5', 'AFIXES', 'ZIF-65', 'ZIF-8', 'BAHGUN03', 'IDIWOH03', 'ITUVEZ']
['ZIF

In [9]:
# Save model
# cnn_model.save("cnn_3-zif8-in-top3-3-zif90-in-top1.h5")

In [29]:
# Test_first=np.load('fft_data14.npy')
# Test_first = Test_first[:7,:2251]
# Test_first = pd.DataFrame(Test_first)
# # print(Test_first.shape)
# with open('norm_T_origin.pickle', 'rb') as f:
#     Test_last = pickle.load(f)
#     Test_last = pd.DataFrame(Test_last)
# Test_last = Test_last.iloc[7:,:]
# # print(Test_last.shape)
# Test = Test_first.append(Test_last,ignore_index=True)
# Test.shape

Test = pd.read_csv('mix_test.csv')

X_test = scaler.transform(Test.iloc[:,1:])
pred = cnn_model.predict(X_test.reshape(14, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)
# print(predicted)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:8]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['JARMEU10', 'DUT-7', 'UGUQOC', 'JARMEU', 'ZIF-67', 'ZIF-71', 'ZIF-65', 'GORQAF']
['IDIWOH', 'ZIF-67', 'JARMEU10', 'DUT-7', 'JARMEU', 'UGUQOC', 'IDIWOH07', 'GORQAF']
['JOSNAG', 'NUMMOX', 'WABVIF04', 'JARMEU10', 'ZIF-75', 'REGJIW', 'TILVOZ', 'JARMEU']
['JARMEU10', 'UGUQOC', 'IDIWOH', 'DUT-23', 'JARMEU', 'HUYKIV', 'QOJTUE', 'ZIF-6']
['AFEHUO', 'ZIF-70', 'QEFNAQ', 'ZIF-71', 'SAJRAW10', 'HUKRIO', 'EDIKEH', 'QAYTOZ']
['AFEHUO', 'ZIF-70', 'QEFNAQ', 'QAYTOZ', 'SAJRAW10', 'HUKRIO', 'JARMEU10', 'ZIF-71']
['AFEHUO', 'ZIF-70', 'SAJRAW10', 'YEFPAA', 'QEFNAQ', 'HUKRIO', 'QAYTOZ', 'XAYMUF']
['ZIF-90', 'MOF-5', 'IDIWOH03', 'ZIF-65', 'ZIF-8', 'DEFKUU', 'JOSNAG01', 'BAHGUN03']
['MOF-5', 'ZIF-90', 'ZIF-8', 'IDIWOH03', 'ZIF-65', 'AFIXES', 'HUYKUH', 'UHAXUW']
['MOF-5', 'ZIF-90', 'ZIF-8', 'UHAXUW', 'AFIXES', 'HUYKUH', 'BAHGUN03', 'PUTYEI']
['MOF-5', 'ZIF-90', 'ZIF-8', 'IDIWOH03', 'UHAXUW', 'ZIF-65', 'AFIXES', 'HUYKUH']
['ZIF-90', 'MOF-5', 'AFIXES', 'ZIF-65', 'ZIF-8', 'BAHGUN03', 'IDIWOH03', 'ITUVEZ']
['ZIF

In [21]:
# Test.to_csv('mix_test.csv')